# Image deblurring

**Authors: M. Ravasi, D. Vargas, I. Vasconcelos**

Welcome to the second session of our **Solving large-scale inverse problems in Python with PyLops** tutorial!

Throughout this tutorial, we aim at developing the following aspects in image processing:

- We connect with the concepts learned in the previous session by defining a simple PyLops operator implementing a convolutional kernel for image processing  
- Learn how to use build-in PyLops operators for image manipulation including, blurring, sharpening, and edge detection
- Demonstrate the versatility of the linear operators when implemented in the solution of inverse problems, in particular, the image deblurring problem
- Analyze the influence of noise in contrast with the performance of different solvers, least-squares, TV-Regularization, and FISTA

## Useful links

- Tutorial Github repository: https://github.com/mrava87/pylops_pydata2020
        
- PyLops Github repository: https://github.com/equinor/pylops

- PyLops reference documentation: https://pylops.readthedocs.io/en/latest/

# Deblurring Images

Compact digital cameras were introduced in the early seventies thanks to the development of CCD sensors allowing the recording and storage of images in digital format. The goal of an image process is to capture an accurate representation of a particular scene. However, any image is influenced by the optical system itself and the overall effect is a blurred image reconstruction, therefore, specialized algorithms performing deblurring need to be implemented in order to achieve a sharper image. Image deblurring is a method that aims at recovering the original sharp-image by removing effect caused by limited aperture, lens aberrations, defocus, and unintended motions. This can be done by defining a mathematical model of the blurring process with the idea of removing from the image the blurring effects.

Traditionally, the main assumption is that the blurring process is linear. When this is the case, a 2-dimensional convolutional model describes the imaging problem. The blurred Image (output) is the result of a point-spread function acting on a target sharp image (input). Such process is described by the following transformation, 

\begin{equation}
g(x, y) = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty} h(x-x_0,y-y_0) f(x_0, y_0) dx_0 dy_0.
\end{equation}

This operation can be discretized as follows

\begin{equation}
g[i,n] = \sum_{j=-\infty}^{\infty} \sum_{m=-\infty}^{\infty} h[i-j,n-m] f[j,m] 
\end{equation}

<img src="figs/psf.png" width="850">

and also implemented in the frequency domain using the convolution theorem

\begin{equation}
G(k_x, k_y) = H(k_x, k_y) F(k_x, k_y).
\end{equation}

Here, the spectrum of the point-spread function $H(k_x, k_y) = \mathscr{F} \{h(x,y)\}$ is called transfer function, similarly, $F(k_x, k_y) = \mathscr{F} \{f(x,y)\}$ is the spectrum of the sharp image. The previous definition is an specific case of the more general problem $\mathbf{y} =  \mathbf{A} \mathbf{x}$ (*forward operation*), where $\mathbf{A}:\mathbb{F}^m \to \mathbb{F}^n$ is a linear operator that maps a vector of size $m$ in the *model space* to a vector of size $n$ in the *data space*, as explained in the previous session. The (*adjoint operation*) is $\mathbf{x} = \mathbf{A}^H \mathbf{y}$. In this context, our deblurring operation is an inverse problem design to find a pseudoinverse, $\mathbf{A}^{-1}$, that aims at removing the effect of the operator $\mathbf{A}$ from the data $\mathbf{y}$ to retrieve the model $\mathbf{x}$, i.e., $\hat{\mathbf{x}} = \mathbf{A}^{-1} \mathbf{y}$. 

This Jupyter Notebook is intended to guide you through the main steps in the implementation of the blurring and deblurring process using the <code>pylops.signalprocessing.Convolve2D</code> operator, in this case, we assume to know the structure of the point-spread function for an specific problem.



In [ ]:
# Run this when using Colab (will install the missing libraries)
# !pip install pylops scooby

In [ ]:
%matplotlib inline

import numpy as np
from skimage import data
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.signal import convolve, correlate
import scooby

from pylops.optimization import leastsquares, sparsity
from pylops.signalprocessing import Convolve2D
from pylops import FirstDerivative, Laplacian
from pylops import LinearOperator
from pylops.utils import dottest

# A simple  deconvolution problem

The first problem we will solve with the aid of pylops is a simple one-dimensional convolution. We start by defining a pylops operator applying convolution with a given impulse response to a model vector. For simplicity, we select a *[ricker](https://en.wikipedia.org/wiki/Mexican_hat_wavelet)* wavelet and used it as a compact filter to be convolved with a 1D input signal. Assuming our input signal (model) is a time series constructed by superposition of three gaussian functions with different time shifts, the forward problem can be interpreted as a weighted sum of the function $f(\tau)$ at the moment $t$ with weights given by $h$.

<img src="figs/convolve_1D.png" width="800">

The resulting signal (data), $g(t)$, is a 'blurred' shifted version of the input model mathematically computed by integration over time according to 

\begin{equation}
g(t) = \int_{-\infty}^{\infty} h(t - \tau) f(\tau) d\tau
\end{equation}

Next, we solve the inverse problem (**deconvolution**) where we ask the question. What model vector would reproduce a given data set when convolved with a specific convolutional kernel?. Here we try to reconstruct an unknown model based on the given data (blue line in the figure) and impulse response (red line in the figure). The expected output of this operation should approximate our target model (black_line in the figure)


**Pylops convolution operator:**
A simple pylops operator is defined as a Child class that inherits from **[`pylops.LinearOperator`](https://pylops.readthedocs.io/en/latest/api/generated/pylops.LinearOperator.html#pylops.LinearOperator)**. We initilize the class by defining at least three atributes in the the class constructor; `shape`, `dtype`, and `explicit`. Next, we difine the forward mode operation through the method `_matvec`, and `_rmatvec` for the adjoint mode.

In [ ]:
# Defining a convolution Pylops operator
class my_convolve(LinearOperator):
    def __init__(self, A, dims, dtype="float64"):
        self.A = A
        self.dims = dims
        self.shape = (np.prod(self.dims), np.prod(self.dims))
        self.dtype = np.dtype(dtype)
        self.explicit = False

    def _matvec(self, x):
        x = np.reshape(x, self.dims)
        y = convolve(x, self.A, mode='same')
        return np.ndarray.flatten(y)

    def _rmatvec(self, x):
        x = np.reshape(x, self.dims)
        y = correlate(x, self.A, mode='same')
        return np.ndarray.flatten(y)

In [ ]:
t_min, t_max, nt = -4.0, 4.0, 10000
f_central = 15
t_delay = 1
t_delay_wav = np.array([0.0, 0.6, 1.2])[:, np.newaxis]

# Time axis
t = np.linspace(t_min, t_max, nt)
sigma = 1 / (np.pi * f_central) ** 2

# Model vector
wav = np.exp(-((t - t_delay_wav) ** 2) / sigma)
model_ = wav[0] - 1.5*wav[1] + wav[2]

# Impulse response - Convolutional Operator
impulse_response = (1 - (2 * (t - t_delay) ** 2) / sigma) * \
    np.exp(-((t - t_delay) ** 2) / sigma)

# Data vector
R_op = my_convolve(A=impulse_response, dims=(nt))
data_ = R_op * model_



# Model reconstruction - Inverse problem
model_reconstructed = sparsity.FISTA(Op=R_op,
                                    data=data_,
                                    eps=1e-1,
                                    niter=200)[0]

# Plotting input vectors
fig, ax = plt.subplots(1, 3, figsize=[10, 3], facecolor='w', edgecolor='k')
ax[0].plot(t, impulse_response, 'r', lw=1.5)
ax[0].set_xlabel('Time - (sec)')
ax[0].set_ylabel('Amplitude')
ax[0].set_xlim([0.8, 1.2])
ax[0].set_title("Operator")

ax[1].plot(t, model_, 'k', lw=1.5)
ax[1].set_xlabel('Time - (sec)')
ax[1].set_xlim([-0.1, 2.5])
ax[1].set_title("Model")

ax[2].plot(t, data_, 'b', lw=1.5)
ax[2].set_xlabel('Time - (sec)')
ax[2].set_xlim([-0.1, 2.5])
ax[2].set_title("Data")
fig.tight_layout()


# Plotting Model reconstruction vectors
fig = plt.figure(figsize=[4, 3], facecolor='w', edgecolor='k')
ax = fig.add_subplot(1, 1, 1)
ax.plot(t, model_, 'k', lw=3, label='Target Model')
ax.plot(t, model_reconstructed, '--g', lw=2, label='Reconstruction')
ax.set_ylabel('Amplitude')
ax.set_xlabel('Time - (sec)')
ax.set_xlim([-0.1, 1.5])
ax.set_title("Model Reconstruction")
ax.legend()

In [ ]:
R_op

# Image filtereing

PyLops comes with multiple build-in operators ready to use. In this section, we are using `pylops.signalprocessing.Convolve2D` to perform convolutions. First, we need to define a compact filter to construct the actual operator and then apply it to a pre-loaded image. Before we do that, let's first see if under the given parameters this linear operator passes the dot test. Once we make sure that this is the case, we can compute the convolution using NumPy matrix-like syntax. 

<img src="figs/filtered_image.png" width="800">

In the cell below, we select the most simple point spread function, the identity kernel. It should not blur the image and the error of the output compared to the input should be very low (to machine precession)

Let's start by uploading an image. Here we use the **scikit-image** module <code>skimage.data</code> containing standard test images. The very first line selects the image of your preference to be used in the next cells, feel free to play around, and even upload your own pictures. 

In [ ]:
image = data.camera()         # Load image from scikit-image
# image = data.binary_blobs()
# image = data.checkerboard()
ny, nx = image.shape

# Identity kernel
kernel = np.zeros([3, 3])
kernel[1, 1] = 1

# Define the PyLops blurring Operator
C_op = Convolve2D(N=ny*nx,
                  h=kernel,
                  offset=(kernel.shape[0]//2, kernel.shape[1]//2),
                  dims=(ny, nx))

# Dot test
dottest(C_op, ny*nx, ny*nx, verb=True)


img_blur = C_op * image.flatten()
img_blur = img_blur.reshape(image.shape)


# PLOTTING
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(image, cmap='gray')
ax[0].set_title('Input image')

ax[1].imshow(img_blur, cmap='gray')
ax[1].set_title('Response to identity')

him = ax[2].imshow(img_blur-image, cmap='gray')
ax[2].set_title('Error output-input')
fig.colorbar(him, ax=ax[2])

In [ ]:
print("image.shape {}".format(image.shape))
C_op

**EX: Write your own kernel.** A very common practice in digital image processing consists of applying filters as a way to highlight specific image features. One of the most useful is known as unsharp mask and is used attenuate low frequencies features while enhancing small contrast changes. A very simple version of this sharpening filter is given by the matrix kernel,

$$ 
\mathbf{A} = 
\begin{bmatrix}
    0 & -1 & 0\\
    -1 & 5 & -1\\
    0 & -1 & 0
\end{bmatrix} 
$$

Define a pylops operator for this mask and apply it to your preferred image. A code snippet for the plotting the input/output images is given below 

<img src="figs/unsharp_mask.png" width="800">

In [ ]:
# %load -s Unsharp_Mask solutions/deblurring_sol.py

In [ ]:
# Unsharp_Mask()

#### Edge detection - Sobel operator

Edge detection is a process that tries to identify areas on an image where strong changes in intensity take place. A simple way of performing this task is by implementing a Sobel operator. This kind of mask approximates the directional derivates of an image along with directions $x$, and $y$. Vertical, $\mathbf{G_x}$, and Horizontal, $\mathbf{G_y}$, gradients are give by the 3x3 kernels

\begin{equation}
    \mathbf{G_x}=
    \begin{bmatrix}
        1 & 2 & 1\\
        0 & 0 & 0\\
        -1 & -2 & -1
    \end{bmatrix} 
   \qquad\text{,}\qquad 
    \mathbf{G_y} =
    \begin{bmatrix}
        1 & 0 & -1\\
        2 & 0 & -2\\
        1 & 0 & -1
    \end{bmatrix}
\end{equation}

note that each mask is designed to detect either horizontal or vertical changes. A Combination of these gradient approximations provides the total gradient magnitude, $\mathbf{G} = \sqrt{\mathbf{G_x}^2 + \mathbf{G_x}^2}$, at an specific pixel of the image.

As we did in the previous exercise, we now implement edge detection by defining vertical and horizontal gradient kernels, apply them to an image, and assemble the gradient magnitude.

In [ ]:
# vertical edge detector
kernel_v = np.array([[1, 0, -1],
                     [2, 0, -2],
                     [1, 0, -1]])

# horizontal edge detector
kernel_h = np.array([[1, 2, 1],
                     [0, 0, 0],
                     [-1, -2, -1]])

# Define the PyLops blurring Operator
Sobel_x_op = Convolve2D(N=ny*nx,
                        h=kernel_h,
                        offset=(kernel_h.shape[0]//2, kernel_h.shape[1]//2),
                        dims=(ny, nx))
Sobel_y_op = Convolve2D(N=ny*nx,
                        h=kernel_v,
                        offset=(kernel_v.shape[0]//2, kernel_v.shape[1]//2),
                        dims=(ny, nx))

Gx = Sobel_x_op * image.flatten()
Gy = Sobel_y_op * image.flatten()
Gx = Gx.reshape(image.shape)
Gy = Gy.reshape(image.shape)

# Gradient magnitude
G = np.sqrt(Gy**2 + Gy**2)

# PLOTTING
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].imshow(image, cmap='gray')
ax[0].set_title('Image')

ax[1].imshow(kernel_v, cmap='gray')
ax[1].set_title('vertical Kernel')

ax[2].imshow(G, cmap='gray')
ax[2].set_title('Image edges')


# Image Blurring

The Gaussian filter is an operator used to suppress noise and reduce the contrast of an image. In this case, an image is rendered smooth under the action of a Gaussian mask by removing the high-frequency components.  Therefore, we can understand a Gaussian blur as a low-pass filter, attenuating the high-frequency components in an image. Similarly, the maximum value of the weighted sum in the convolution located around the center of the distribution decreases as a function of distance. Mathematically the two-dimensional Gaussian function is defined as

\begin{equation}
G(x,y) = \frac{1}{2 \pi \sigma ^{2}} e^{- \frac{x^{2} + y^{2}}{2 \sigma ^{2}}}
\label{2dgaussian}
\end{equation}

where the standard deviation of the kernel is given for each axis and determines the effective extension of the filter.

<img src="figs/blur_model.png" width="800">

In the following cell, we will prepare a data vector represented by a Gaussian-blurred image as preparation for the next section, where we will use the smoothed image together with the given kernel and try to remove the blurring effects by solving a deconvolution problem 

In [ ]:
# Play around with this parameters
mx, my = 25, 15                 # 25, 15
sigma_x, sigma_y = 4.5, 2.5     # 4.5, 2.5

x = np.linspace(-mx//2, mx//2, mx)
y = np.linspace(-my//2, my//2, my)[..., None]
kernel_gauss = 1/(2*np.pi*sigma_x*sigma_y) * \
    np.exp(-(x**2/(2*sigma_x**2) + y**2/(2*sigma_y**2)))

# Define the PyLops blurring Operator
Gauss_op = Convolve2D(N=ny*nx,
                      h=kernel_gauss,
                      offset=(kernel_gauss.shape[0]//2,
                              kernel_gauss.shape[1]//2),
                      dims=(ny, nx))

img_gauss = Gauss_op * image.flatten()
img_gauss_ = img_gauss.reshape(image.shape)

# image spectrum
image_shift = np.fft.fftshift(np.fft.fft2(image))
image_spectrum = np.abs(image_shift)

# PSF spectrum - Transfer function
kernel_gauss_shift = np.fft.fftshift(np.fft.fft2(kernel_gauss))
kernel_gauss_spectrum = np.abs(kernel_gauss_shift)

# blurred-image spectrum
img_gauss_shift = np.fft.fftshift(np.fft.fft2(img_gauss_))
img_gauss_spectrum = np.abs(img_gauss_shift)

# PLOTTING
fig, ax = plt.subplots(2, 3, figsize=(9, 6))

ax[0, 0].imshow(image, cmap='gray')
ax[0, 0].set_title('Image')

ax[0, 1].imshow(kernel_gauss, cmap='gray')
ax[0, 1].set_title('Gaussian Kernel')

him2 = ax[0, 2].imshow(img_gauss_, cmap='gray')
ax[0, 2].set_title('Blurred Image')

ax[1, 0].imshow(image_spectrum, cmap='gray', norm=LogNorm(vmin=10))
ax[1, 0].set_title('Image Spectrum')

ax[1, 1].imshow(kernel_gauss_spectrum, cmap='gray')
ax[1, 1].set_title('Gaussian Kernel Spectrum')

him2 = ax[1, 2].imshow(img_gauss_spectrum, cmap='gray', norm=LogNorm(vmin=10))
ax[1, 2].set_title('Blurred Image Spectrum')
fig.tight_layout()

# Image Deblurring 

Finally, we consider a standard deblurring case. Here we assume a picture was taken in real life; therefore, a whole set of potential artifacts are present in it. Our mission is to try and reconstruct a shaper image representation of the scene by removing the blurring effects. Luckily, we assume to have some information on the optical system and decided to implement an inverse problem. Depending on the application, different solvers are available, below we will compare the performance of some well-known algorithms to deblur the image and compare it with its original version.

- least squares inversion
- least squares - regularized inversion 
- TV inversion
- FISTA - Fast Iterative Shrinkage-Thresholding Algorithm

A set of ready to use solvers are implemented in the PyLops module `pylops.optimization.leastsquares` and `pylops.optimization.sparsity`.

**least squares inversion**
\begin{equation}
J= ||\mathbf{d} - \mathbf{A} \mathbf{m}||_2^2
\end{equation}

In [ ]:
# least squares inversion
deblur_l2 = leastsquares.NormalEquationsInversion(Op=Gauss_op,
                                                  Regs=None,
                                                  data=img_gauss,
                                                  maxiter=40)
# Reshape image
deblur_l2 = deblur_l2.reshape(image.shape)

**least squares - regularized inversion**
\begin{equation}
J= ||\mathbf{d} - \mathbf{A} \mathbf{m}||_2^2 + ||\nabla \mathbf{m}||_2^2
\end{equation}

In [ ]:
# L2 regularization term - Second derivative
D2op = Laplacian(dims=(ny, nx), dtype=np.float)

# least squares - regularized inversion
deblur_l2_reg = leastsquares.RegularizedInversion(Op=Gauss_op,
                                                  Regs=[D2op],
                                                  data=img_gauss,
                                                  epsRs=[1e0],
                                                  show=True,
                                                  **dict(iter_lim=20))
# Reshape image
deblur_l2_reg = np.real(deblur_l2_reg.reshape(image.shape))

**TV inversion**
\begin{equation}
J= ||\mathbf{d} - \mathbf{A} \mathbf{m}||_2^2 + ||\mathbf{D}_x  \mathbf{m}||_1 + ||\mathbf{D}_y \mathbf{m}||_1
\end{equation}

In [ ]:
# L1 regularization term  - First derivative
Dop = [FirstDerivative(ny * nx, dims=(ny, nx), dir=0),
       FirstDerivative(ny * nx, dims=(ny, nx), dir=1)]

# TV inversion
deblur_tv = sparsity.SplitBregman(Op=Gauss_op,
                                  RegsL1=Dop,
                                  data=img_gauss,
                                  niter_outer=10,
                                  niter_inner=5,
                                  mu=1.8,
                                  epsRL1s=[1e-1, 1e-1],
                                  tol=1e-4,
                                  tau=1.,
                                  ** dict(iter_lim=5, damp=1e-4, show=True))[0]
# Reshape image
deblur_tv = deblur_tv.reshape(image.shape)

**FISTA inversion**
\begin{equation}
J= ||\mathbf{d} - \mathbf{A} \mathbf{m}||_2^2 + \epsilon||\mathbf{m}||_p
\end{equation}

In [ ]:
# FISTA inversion
deblur_fista = sparsity.FISTA(Op=Gauss_op,
                              data=img_gauss,
                              eps=1e-1,
                              niter=100,
                              show=True)[0]
# Reshape image
deblur_fista = deblur_fista.reshape(image.shape)

In [ ]:
# PLOTTING
fig, ax = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(12, 8))

ax[0, 0].imshow(image, aspect='auto', cmap='gray')
ax[0, 0].set_title('Original Image')

ax[0, 1].imshow(deblur_l2, aspect='auto', cmap='gray')
ax[0, 1].set_title('LS-Inversion')

ax[0, 2].imshow(deblur_l2_reg, aspect='auto', cmap='gray')
ax[0, 2].set_title('Regularized LS-Inversion')

ax[1, 0].imshow(img_gauss_, aspect='auto', cmap='gray')
ax[1, 0].set_title('Blurred Image')

ax[1, 1].imshow(deblur_tv, aspect='auto', cmap='gray')
ax[1, 1].set_title('TV-Inversion')

ax[1, 2].imshow(deblur_fista, aspect='auto', cmap='gray')
ax[1, 2].set_title('FISTA Inversion')
fig.tight_layout()

# Image Deblurring - The effect of noise Noise

We have so far discussed the blurring of images. In practical image processing, observed images are commonly polluted with noise. The nature of the noise is influenced by multiple sources, and mathematically can be linear, nonlinear, multiplicative, and additive. In this exercise, we consider common additive noise, which can be included in our linear model by a term perturbing the blurred image 

$$
\mathbf{B} = \mathbf{B_{blur}} + \mathbf{N}
$$

**EX: Adding noise.** Add Gaussian white noise to the blurred image with a given standard deviation and perform image deblurring to evaluate the performace of different solvers

In [ ]:
# %load -s Noisy_Inversion solutions/deblurring_sol.py

In [ ]:
# Noisy_Inversion()

# Recap

In this tutorial we have learned to:

- Define a simple PyLops operator implementing a convolutional kernel for image processing  
- Use build-in PyLops operators for image manipulation including
- Solve the image deblurring problem using different inverse problems schemes
- Denoise a blurred image with the help of least-squares, TV-Regularization, and FISTA


In [ ]:
scooby.Report(core='pylops')